In [101]:
import pandas as pd
import numpy as np

data = pd.read_csv('/home/nika/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')
df = data.copy()
df.head()

/tmp/ipykernel_3350/3543565571.py:4: DtypeWarning: Columns (24,25,26,35,43,53,59,63,74,75,85,89,108,110,116,120,122,124,136,141,147,150,151,162,163,177,178,180,195,214,221,226,234,241,246,259,266,270,271,273,278,283,287,290) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/nika/code/marcnaweb/car_recommendation_engine/raw_data/car_files_4c_en.csv')


,Unnamed: 0.1,Unnamed: 0,Hot air,Volumetric anti -first alarm,Harman Kardon Sound System,Track change alert,Carbonaceramic brake disc,Arconditional Zones 2,Reclinarinar rear seat,Maximum rotation,...,Folding rear seat,Free wheel,Shaker Pro Audio Sound System,Maximum speed electric mode,Electronic noise cancellation,Steering wheel adjustment in depth,Facial recognition camera,Carcode,Perimeter anti theft alarm,Track centralization assistant
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN


## Too many features with NaN's, remove columns which have more then 15000 Nan'n in column

In [102]:
filtered_columns = df.columns[df.isna().sum() < 15000]
filtered_df = df[filtered_columns]
# filtered_df

## Drop some rows with a lot of Nan's

In [103]:
test_filtered_df =filtered_df.dropna(subset='Acceleration')
# test_filtered_df

## Check how much % is NaN in every column and check Dtype

In [104]:
nan_percentage_per_column = test_filtered_df.isnull().mean() * 100

# Create a new DataFrame with column names and NaN percentages and Dtypes
nan_info_df = pd.DataFrame({
    'Column Name': nan_percentage_per_column.index,
    'NaN Percentage': nan_percentage_per_column.values,
    'Data Type': test_filtered_df.dtypes
})

# Display the new DataFrame
nan_info_df # To check whole list: nan_info_df[0:50] and nan_info_df[50:100]

,Column Name,NaN Percentage,Data Type
Unnamed: 0.1,Unnamed: 0.1,0.000000,int64
Hot air,Hot air,46.233340,object
Acceleration,Acceleration,0.000000,object
Length,Length,0.000000,object
Maximum torque,Maximum torque,0.709871,object
...,...,...,...
Year,Year,0.125555,float64
Radio,Radio,49.565385,object
Folding rear seat,Folding rear seat,52.796021,object
Carcode,Carcode,0.000000,int64


## After analysing all features, decided to remove some of them.
Neither they don't give information, or they are hard to preprocess.


In [107]:
column_to_remove = ['Unnamed: 0.1','Assistance','Carcode','Aspiration', 'Rear tires', 'Spare tire', 'Front tires',
                    'Urban autonomy', 'Generation', 'Front suspension', 'Rear suspension', 'Coupling', 'Valve command',
                    'Places', 'Disposition', 'Cylinders', 'Elastic element', 'Ipva R', 'Frontal area A', 'Engine code',
                    'Traction', 'Installation', 'Road autonomy', 'Engine power supply', 'Engine control activation',
                    'Gear change code', 'Corrected frontal area', 'Platform', 'Tuching']


In [108]:
new_filtered_df = test_filtered_df.drop(columns=column_to_remove)
len(new_filtered_df.columns)

70

# Preprocessing Features

## Preprocessing All features with [Nan, standart equipment, optinal equipment]
saved in: columns_to_transform ---------
Nan = 0 ----------------
standart equipment, optinal equipment = 1

In [109]:
preprocessing_df = new_filtered_df.copy()

columns_to_transform = ['Hot air', 'Rev counter', 'Assisted direction', 'ABS brakes', 'Rear window', 'Central locking of the doors',
                        'Headrest for all occupants', 'Electric rearview mirror adjustment', 'Air conditioning',
                        'Bluetooth connection', 'Frontal Airbags', 'Steering wheel adjustment height',
                        'Electric front window control', 'Multifunctional steering wheel', 'Driver s seat with height adjustment',
                        'On board computer', 'Light in the trunk', 'Alloy wheels', 'USB connection', 'Radio',
                        'Folding rear seat', 'Perimeter anti theft alarm', 'Cooling liquid thermometer']

# Replace NaN with 0 and other values with 1
preprocessing_df[columns_to_transform] = preprocessing_df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


/tmp/ipykernel_3350/589099134.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  preprocessing_df[columns_to_transform] = preprocessing_df[columns_to_transform].applymap(lambda x: 0 if pd.isnull(x) else 1)


## Preprocessing [Acceleration]

In [110]:
# Preprocess Acceleration (0100 km/h 3,8 s = 3.8)
preprocessing_df['Acceleration'] = preprocessing_df['Acceleration'].str.extract(r'(\d+\,\d+)')

# Replace ',' with '.' and convert to numeric
preprocessing_df['Acceleration'] = preprocessing_df['Acceleration'].str.replace(',', '.').astype(float)
# Rename Column
# preprocessing_df = preprocessing_df.rename(columns={'Acceleration': 'Acceleration 0100 km/h in S'})

## Preprocessing [All features with strings]

In [9]:
# Rename columns
# preprocessing_df = preprocessing_df.rename(columns={'Length': 'Length mm',
#                                                     'Width': 'Width mm',
#                                                     'Maximum torque': 'Maximum torque kgfm'})

In [111]:
def extract_float_value(value):
    try:
        if isinstance(value, float):
            return value
        else:
            float_value = value.split()[0].replace(',', '.')
            return float(float_value)
    except (ValueError, IndexError):
        return np.nan

for column in preprocessing_df[['Weight/Torque', 'Weight', 'Weight/power', 'Max power regime.', 'Cylinder diameter',
                      'Fuel tank', 'Specific power', 'Maximum power', 'Length', 'Maximum torque', 'Width', 'Height',
                      'Specific torque', 'Minimum height from the ground', 'Piston course', 'Front gauge', 'Displacement',
                      'Turns diameter', 'Rear gauge', 'Length between the axis', 'Maximum speed', 'Road consumption',
                      'Max torque regime', 'Car payload', 'Sidewall height', 'Unit displacement', 'Trunk', 'Urban',
                      'Guarantee', 'Reader score', 'Compression ratio', 'Drag coefficient', 'Price R', 'Devaluation',
                      'CNW Index']]:
    preprocessing_df[column] = preprocessing_df[column].apply(extract_float_value)


In [112]:
len(preprocessing_df.columns)

70

## Creating a pipeline

In [130]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [113]:
# WORKING PIPELINE
columns_to_encode = ['Propulsion', 'Car gearbox', 'Fuel', 'Settings', 'Car size']

# Create Pipeline
pipeline_onehot = Pipeline([
    ('encoder', ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), make_column_selector(dtype_include="object"))]  # Apply one-hot encoding to the specified columns
    , remainder='passthrough')),  # Pass through any remaining columns unchanged
])

In [114]:
def binarize_column(column):
    binarizer = LabelBinarizer()
    return binarizer.fit_transform(column)

In [115]:
numeric_columns = preprocessing_df.select_dtypes(include=['number'])

numeric_column_names = numeric_columns.columns.tolist()
len(numeric_column_names)

64

In [131]:
# pipelne for whole df // WORKING !!!!

pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('onehot', OneHotEncoder(), columns_to_encode),  # Apply one-hot encoding to specified columns
        ('binarize', FunctionTransformer(binarize_column, validate=False), ['Provenance']),  # Binarize the binary column
        ('imputer', SimpleImputer(strategy='median'), numeric_column_names),  # Impute missing values
    ], remainder='passthrough')),
    ('scaler', MinMaxScaler())
])

In [132]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot', OneHotEncoder(),
                                                  ['Propulsion', 'Car gearbox',
                                                   'Fuel', 'Settings',
                                                   'Car size']),
                                                 ('binarize',
                                                  FunctionTransformer(func=<function binarize_column at 0x7f171e3b1120>),
                                                  ['Provenance']),
                                                 ('imputer',
                                                  SimpleImputer(strategy='median'),
                                                  ['Hot air', 'Acceleration',
                                                   'Length', 'Maxi...
                                                   'Drag coefficient',
                                                   'Central locking of the '
                                                   'doors',
                                                   'Height',
                                                   'Valves per cylinder',
                                                   'Specific torque',
                                                   'Minimum height from the '
                                                   'ground',
                                                   'Piston course', 'Weight',
                                                   'Price R',
                                                   'Headrest for all occupants',
                                                   'Front gauge',
                                                   'Electric rearview mirror '
                                                   'adjustment',
                                                   'Weight/power',
                                                   'Displacement',
                                                   'Cooling liquid thermometer',
                                                   'Max power regime.',
                                                   'Turns diameter', ...])])),
                ('scaler', MinMaxScaler())])

In [134]:
testing_pipe_df = preprocessing_df.copy()
testing_pipe_df_end = pipeline.fit_transform(testing_pipe_df)
testing_pipe_df_end = pd.DataFrame(testing_pipe_df_end)
testing_pipe_df_end.shape

# check if there are nan valuues
# has_nan = testing_pipe_df_end.isna().any().any()

# if has_nan:
#     print("DataFrame contains NaN values")
# else:
#     print("DataFrame does not contain any NaN values")


(20708, 100)

### old (no need anymore)

In [39]:
# create dataframe with features we want to encode
onehotencoded_df = preprocessing_df[columns_to_encode]
# Fit and transform the data
onehotencoded_df = pipeline_onehot.fit_transform(onehotencoded_df)
# Get column names
encoded_column_names = pipeline_onehot.named_steps['encoder'].get_feature_names_out(columns_to_encode)
# Remove onehot__ from the columns (example: onehot__Propulsion_combustion TRANSFORM: Propulsion_combustion )
encoded_column_names = [column.replace('onehot__', '') for column in encoded_column_names]
# Transform to pd dataframe with column names
onehotencoded_df = pd.DataFrame(onehotencoded_df, columns = encoded_column_names)
# Remove columns that we encoded from main df
remaining_columns_df = preprocessing_df.drop(columns=columns_to_encode)
# Combine the one-hot encoded features with the remaining columns of the original DataFrame
final_df = pd.concat([remaining_columns_df, onehotencoded_df], axis=1)
final_df.columns

len(final_df.columns)


# We had 71 columns at the start, we encoded 5 columns and as a result we received 35 columns,
# 71+35 = 106. now we delete original 5 columns, so 106-5=101. Everything is correct.


101

In [17]:
final_df.shape

(21642, 101)

In [18]:
test_df = final_df.dropna(subset=['Hot air'])
test_df

,Hot air,Acceleration,Length,Maximum torque,Width,Rev counter,Compression ratio,Assisted direction,CNW Ranking,ABS brakes,...,Settings_SUV,Settings_Sedan,"Settings_Turkey, hen",Settings_Van,Car size_Compact and portable,Car size_Coupe,Car size_Grande,Car size_Medium,Car size_Medium Compact,Car size_Subcompact
0,0.0,3.8,4580.0,66.3,2045.0,0.0,10.71,0.0,20549.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,15.2,3644.0,8.5,1548.0,0.0,9.51,0.0,14268.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,10.6,4177.0,17.3,1709.0,0.0,9.81,0.0,12266.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,8.5,4149.0,21.4,1735.0,0.0,9.51,0.0,16041.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,9.5,4187.0,16.7,1741.0,0.0,10.31,0.0,9142.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23881,1.0,10.7,4015.0,17.5,1720.0,1.0,10.51,1.0,63.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23882,1.0,10.7,4015.0,17.5,1720.0,1.0,10.51,1.0,54.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23890,1.0,10.7,4015.0,17.5,1720.0,1.0,10.51,1.0,50.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23902,1.0,14.9,5413.0,34.7,2050.0,1.0,161.00,1.0,1073.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
final_df.isna()
nan_percentage = (test_df.isna().sum() / len(test_df)) * 100
nan_percentage[0:20]


Hot air                            0.000000
Acceleration                      17.046552
Length                             0.000000
Maximum torque                     0.709871
Width                              0.000000
Rev counter                        0.000000
Compression ratio                  1.265212
Assisted direction                 0.000000
CNW Ranking                        0.000000
ABS brakes                         0.000000
Rear window                        0.000000
Tuching                           42.080355
Weight/Torque                      0.000000
Guarantee                          0.000000
Drag coefficient                  37.328569
Central locking of the doors       0.000000
Height                             0.000000
Valves per cylinder                0.714700
Specific torque                    0.709871
Minimum height from the ground    45.484837
dtype: float64

In [63]:
pipeline_boolean = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute NaN values with the most frequent value
    ('encoder', OneHotEncoder(drop='if_binary'))  # One-hot encode the boolean column
])


In [100]:
preprocessing_df['Tuching']
# Carcode

0               NaN
1        Mechanical
2             Hydro
4             Hydro
5             Hydro
            ...    
23881    Mechanical
23882    Mechanical
23890    Mechanical
23902           NaN
23903           NaN
Name: Tuching, Length: 20708, dtype: object

In [69]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import FunctionTransformer

def binarize_column(column):
    binarizer = LabelBinarizer()
    return binarizer.fit_transform(column)

pipeline = Pipeline([
    ('binarizer', FunctionTransformer(binarize_column, validate=False))
])

# Fit and transform the pipeline to the DataFrame
binarized_data = pipeline.fit_transform(preprocessing_df[['Provenance']])

# Create a DataFrame from the binarized data
binarized_df = pd.DataFrame(binarized_data, columns=["Provenance_binary"])

# Print the binarized DataFrame
print(binarized_df)

       Provenance_binary
0                      0
1                      1
2                      1
3                      1
4                      1
...                  ...
20703                  1
20704                  1
20705                  1
20706                  0
20707                  0

[20708 rows x 1 columns]
